In [1]:
import numpy as np

from collections import deque

from PIL import Image
import imageio

from copy import deepcopy

import matplotlib.pyplot as plt
from matplotlib.animation import ArtistAnimation
%matplotlib inline

In [2]:
def process_input(data):
    nums = []
    for line in data:
        nums.append(list(line))
    return np.array(nums).astype(int)

def get_background(data):
    terrain = [[255,255,229],[255,247,188],[254,227,145],
               [254,196, 79],[251,154, 41],[236,112, 20],
               [204, 76,  2],[153, 52,  4],[102, 37,  6],
               [114, 30, 23]]
    frames = []
    background = np.ones((data.shape[0],data.shape[1],3), dtype=int)
    
    for i in range(0, len(data)):
        for j in range(0, len(data[i])):
            background[i,j] = terrain[data[i,j]]
            
    image = Image.fromarray(background.astype('uint8'), mode='RGB')
    image = image.resize((100*4,100*4), resample=Image.NEAREST)
    image.save('./frames/day9_0000.png')
    frames.append(np.array(image))
    
    return background, frames

def get_lowest(data, background, frames):
    #Check if current position is lower than all neighbours
    low_point = np.zeros_like(data, dtype=bool)
    marker = [136,136,136]
    lowest = [ 78,178,101]
    count = 1
    
    for i in range(0, len(data)):
        for j in range(0, len(data[i])):
            
            bg = deepcopy(background)
            bg[i,j] = marker
            image = Image.fromarray(bg.astype('uint8'), mode='RGB')
            image = image.resize((100*4,100*4), resample=Image.NEAREST)
            image.save('./frames/day9_'+str(count).zfill(4)+'.png')
            frames.append(np.array(image))
            count += 1
            
            tgt = data[i,j]
            tst = []
            if i > 0:
                tst.append(data[i-1,j])
            if i < len(data)-1:
                tst.append(data[i+1,j])
            if j > 0:
                tst.append(data[i,j-1])
            if j < len(data[i])-1:
                tst.append(data[i,j+1])
                
            low = True
            for t in tst:
                if tgt >= t:
                    low = False
                    break
            if low:
                background[i,j] = lowest
            low_point[i,j] = low
            
    return low_point, background, frames, count

def good_pos(data, pos):
    if pos[0] < 0 or pos[0] > len(data)-1:
        return False
    elif pos[1] < 0 or pos[1] > len(data[0])-1:
        return False
    elif data[pos[0],pos[1]] == 9:
        return False
    return True

def BFS(data, low_points, background, frames, count, low_coord=None):
    if low_coord == None:
        low_coord = np.where(low_points)
        colour = [ 82,137,199]
    else:
        colour = [230, 85, 24]
    
    #Breadth-First Search from lowest points
    sizes = []
    dx = [-1,1]
    dy = [-1,1]
    for b in range(0, len(low_coord[0])):
        start = (low_coord[0][b], low_coord[1][b])
        queue = deque([start])
        basin = {start: b}
        
        while len(queue):
            cur_pos = queue.popleft()
            
            background[cur_pos[0],cur_pos[1]] = colour
            image = Image.fromarray(background.astype('uint8'), mode='RGB')
            image = image.resize((100*4,100*4), resample=Image.NEAREST)
            image.save('./frames/day9_'+str(count).zfill(4)+'.png')
            frames.append(np.array(image))
            count += 1
            
            for i in range(0, 2):
                #move in x
                nxt_pos = (cur_pos[0]+dx[i], cur_pos[1])
                if good_pos(data, nxt_pos) and nxt_pos not in basin.keys():
                    queue.append(nxt_pos)
                    basin[nxt_pos] = b
                #move in y
                nxt_pos = (cur_pos[0], cur_pos[1]+dy[i])
                if good_pos(data, nxt_pos) and nxt_pos not in basin.keys():
                    queue.append(nxt_pos)
                    basin[nxt_pos] = b
        sizes.append(len(basin.keys()))
        
    idxs = np.argsort(sizes)
    return frames, count, [low_coord[0][idxs[-3:]],low_coord[1][idxs[-3:]]]

In [3]:
inpt = np.genfromtxt('day9_input.txt', dtype=str, delimiter='\n')
print('process_input')
data = process_input(inpt)
print('get_background')
background, frames = get_background(data)
print('get_lowest')
low_point, background, frames, count = get_lowest(data, background, frames)
print('BFS, one')
frames, count, low_coord = BFS(data, low_point, background, frames, count)
print('BFS, two')
frames, count, low_coord = BFS(data, low_point, background, frames, count, low_coord)
print('replicate')
for i in range(0, 60*4):
    image = Image.fromarray(frames[-1].astype('uint8'), mode='RGB')
    image.save('./frames/day9_'+str(count).zfill(4)+'.png')
    frames.append(np.array(image))
    count += 1

process_input
get_background
get_lowest
BFS, one
BFS, two
replicate


In [4]:
print('save')
imageio.mimsave('./day9-vis.gif', frames[::4], duration=1/60)

save
